# Coursera - IBM Data Science capstone project

## This project will segment and cluster the neighbourhoods in Toronto

***
### Step 1: Use this Notebook to build the code to scrape Wikipedia for the Toronto postcodes, boroughs, and neighbourhoods:

The data can be seen in a wikipedia table at this link:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Scrape the table of postal codes and transform the data into a pandas dataframe.

To scrape the webpage:
* Use the BeautifulSoup library to extract the table as html from the page.
* Use pandas read_html() to convert the table as html into a pandas dataframe.


To create the dataframe:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice  and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

Note: This code shows how to extract data from a webpage. Postal codes are fairly static data, so this code would only be used to create a file once. The file would be saved as a config file or reference datafile that's incorporated into a project. Relying on code to download data from a crowdsourced page would make an app fragile to changes, so a best practice is to store the data and only refresh as needed.

<B>Submit a link to your Notebook on your Github repository. (10 marks)</B>


            

In [20]:
import pandas as pd

#import url requests and BeautifulSoup4
import requests
from bs4 import BeautifulSoup

#Use BeautifulSoup to read the wiki page and extract just the html of the table
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content, 'html.parser') # or 'lxml')
html_tables = soup.findAll("table", {"class": ["sortable", "plainrowheaders"]})

#Use pandas read_html to convert the html table dat into a pandas dataframe
df_wiki = pd.read_html(str(html_tables))[0]

df_wiki.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Now we have a dataframe that is ready for cleaning and condensing

In [21]:
#Check the shape and number of entries, then remove all the post codes that are not assigned to a borough.
print("Shape: " ,df_wiki.shape)
print("Borough Not assigned: ", len(df_wiki[df_wiki["Borough"]=="Not assigned"]))
df_cleaned = df_wiki[df_wiki["Borough"]!="Not assigned"]

#Check for unassigned neighborhoods. (0 in this data)
print("Shape cleaned: ", df_cleaned.shape)
print("Neighbourhood Not assigned: ", len(df_cleaned[df_cleaned["Neighbourhood"]=="Not assigned"]))

df_cleaned.head()

Shape:  (287, 3)
Borough Not assigned:  77
Shape cleaned:  (210, 3)
Neighbourhood Not assigned:  0


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [22]:
#Group by Postcode and Borough to see all Neighboorhoods in the postcode, then append neighbourhood names
df_pc = pd.DataFrame(df_cleaned.groupby(['Postcode','Borough'])['Neighbourhood'].apply(", ".join))

#Wow. The pythonic way!  I almost did for loops instead!

#Shape is (103,1) because of multi index. reset and check again to confirm shape (103,3)
print("Shape with groupby index: ", df_pc.shape)
df_pc.reset_index(inplace=True)
print("Shape with reset index: ", df_pc.shape)

#Now let's display the dataframe
df_pc.head()

Shape with groupby index:  (103, 1)
Shape with reset index:  (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 1 completed!  
The dataframe is ready for use in the next step.


***
### Step 2: Add Latitude and Longitude for each postcode 

The suggested geo source is Google's geocoder, or a csv file if geocoder is not available. 

geocoder is currently down.  Use the csv file instead: https://cocl.us/Geospatial_data

Note: A far better source of geo location data on neighbourhoods is available directly from the Toronto Open Data website: 
https://open.toronto.ca/dataset/neighbourhoods/
The file can be downloaded manually or the link for a csv file can be copied and used in code.
https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=csv&projection=4326
The only issue is that it does not include postcode. The data could be merged with the wiki postcode data, or the project redesigend to use this data directly and eliminate the need for both wikipedia and the postcode.

Merge the geo (lat/lon) dataframe with the previous postalcode neighbourhood dataframe.

<B>Once the consolidated dataframe is created, submit a link to the new Notebook on your Github repository. (2 marks)</B>


In [23]:
#Get the geo data for postcodes
df_geo = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
#Fix joining columns
df_geo.rename(columns={"Postal Code": "PostalCode"}, inplace=True) #, errors="raise")
df_geo.head()
df_pc.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df_pc.head()

#Merge the two dataframes on PostalCode
df_toronto = pd.merge(df_pc, df_geo, on='PostalCode')
df_toronto.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Step 2 completed!  
The merged dataframe is ready for use in the next step.


***
### Step 3: Explore and cluster by neighbourhood

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

 * to add enough Markdown cells to explain what you decided to do and to report any observations you make.
 * to generate maps to visualize your neighborhoods and how they cluster together.

<B>Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)</B>

***
#### Take the 40 northernmost postcode areas (neighbourhood groups)

In [36]:
df_north40 = df_toronto.sort_values(by=['Latitude'],ascending=False).head(40)
df_north40

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389
12,M1S,Scarborough,Agincourt,43.794200,-79.262029
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
19,M2K,North York,Bayview Village,43.786947,-79.385975
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
24,M2R,North York,Willowdale West,43.782736,-79.442259


#### Put credentials in a hidden cell.

In [40]:
# The code was removed by Watson Studio for sharing.

Your credentails were loaded. (Cell is hidden)


## 2. Explore the 40 Northmost Neighborhoods in Toronto Area

#### Let's create a function to repeat the same process to all the neighborhoods

In [42]:
#Increase radius because some locations are rural.

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("Calling FourSquare for nearest venues to: ", name)
          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #print(url)  #NOOOOO! Contains credentials
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #results = requests.get(url).json()['response']['venues']


        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    
    #print(venues_list)
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

    #v['venue']['name'], 
    #v['venue']['location']['lat'], 
    #v['venue']['location']['lng'],  
    #v['venue']['categories'][0]['name']) for v in results])





Upper Rouge
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.836124700000006,-79.20563609999999&radius=2000&limit=100
Agincourt North, L'Amoreaux East, Milliken, Steeles East
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.8152522,-79.2845772&radius=2000&limit=100
Rouge, Malvern
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=2000&limit=100
Hillcrest Village
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.80

NameError: name 'noirth40_venues' is not defined

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *north40_venues*.

In [57]:
north40_venues = getNearbyVenues(names=df_north40['Neighbourhood'],
                                   latitudes=df_north40['Latitude'],
                                   longitudes=df_north40['Longitude']
                                  )

Upper Rouge
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.836124700000006,-79.20563609999999&radius=2000&limit=100
Agincourt North, L'Amoreaux East, Milliken, Steeles East
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.8152522,-79.2845772&radius=2000&limit=100
Rouge, Malvern
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=2000&limit=100
Hillcrest Village
https://api.foursquare.com/v2/venues/explore?&client_id=CVWIL5R3DRU3JIBXHD3RQJCQSGKQLZHDQY5HCGWQJVTPA4DD&client_secret=5DRMASF0NAAK5WKKTNQBILMIRMSABMAQ0T0TBV5GUTHIEYDS&v=20180605&ll=43.80

#### Let's check the size of the resulting dataframe

In [45]:
print(north40_venues.shape)
north40_venues.head()


(2884, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Upper Rouge,43.836125,-79.205636,Rouge Park - Finch Meander Trail,43.824850,-79.193596,Trail
1,Upper Rouge,43.836125,-79.205636,SKINlight Laser Clinic,43.819209,-79.205106,Health & Beauty Service
2,Upper Rouge,43.836125,-79.205636,Taj supermarket,43.818746,-79.210172,Grocery Store
3,Upper Rouge,43.836125,-79.205636,Cedar Brae Golf and Country Club,43.839154,-79.230089,Golf Course
4,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,Fahmee Bakery & Jamaican Foods,43.810170,-79.280113,Caribbean Restaurant


Let's check how many venues were returned for each neighborhood

In [47]:
north40_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",99,99,99,99,99,99
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",52,52,52,52,52,52
"Bathurst Manor, Downsview North, Wilson Heights",58,58,58,58,58,58
Bayview Village,49,49,49,49,49,49
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
"CFB Toronto, Downsview East",100,100,100,100,100,100
Cedarbrae,100,100,100,100,100,100
"Clarks Corners, Sullivan, Tam O'Shanter",82,82,82,82,82,82


In [48]:
print('There are {} uniques categories.'.format(len(north40_venues['Venue Category'].unique())))

There are 228 uniques categories.


## 3. Analyze Each Neighborhood

In [50]:
# one hot encoding
north40_onehot = pd.get_dummies(north40_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north40_onehot['Neighbourhood'] = north40_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [north40_onehot.columns[-1]] + list(north40_onehot.columns[:-1])
north40_onehot = north40_onehot[fixed_columns]

north40_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Upper Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Upper Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
print(north40_onehot.shape)

north40_grouped = north40_onehot.groupby('Neighbourhood').mean().reset_index()
north40_grouped

(2884, 229)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.020000,0.000000,0.120000,0.000000,0.00,0.010000,0.080000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00

#### Let's confirm the new size

In [52]:
north40_grouped.shape

(40, 229)

#### Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in north40_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = north40_grouped[north40_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's copy a function to sort the venues in descending order.

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then let's create the new dataframe and display the top 10 venues for each neighborhood.

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = north40_grouped['Neighbourhood']

for ind in np.arange(north40_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north40_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Cantonese Restaurant,Indian Restaurant,Bakery,Breakfast Spot,Shopping Mall
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Dessert Shop,Gas Station,Vietnamese Restaurant,Korean Restaurant,Bakery,Park
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Park,Spa,Skating Rink,Flea Market,Gas Station
3,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Sandwich Place,Park,Convenience Store,Restaurant,Bank,Gas Station,Deli / Bodega,Pharmacy
4,Bayview Village,Chinese Restaurant,Park,Coffee Shop,Pharmacy,Bank,Clothing Store,Café,Gas Station,Grocery Store,Liquor Store


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [98]:
# set number of clusters
kclusters = 5

north40_grouped_clustering = north40_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north40_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 3, 2, 4, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [99]:
neighbourhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ethnic restaurants galore,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Cantonese Restaurant,Indian Restaurant,Bakery,Breakfast Spot,Shopping Mall
1,Ethnic restaurants galore,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Dessert Shop,Gas Station,Vietnamese Restaurant,Korean Restaurant,Bakery,Park
2,Coffee shops everywhere,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Park,Spa,Skating Rink,Flea Market,Gas Station
3,Coffee shops everywhere,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Sandwich Place,Park,Convenience Store,Restaurant,Bank,Gas Station,Deli / Bodega,Pharmacy
4,Ethnic restaurants galore,Bayview Village,Chinese Restaurant,Park,Coffee Shop,Pharmacy,Bank,Clothing Store,Café,Gas Station,Grocery Store,Liquor Store
5,Coffee shops everywhere,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Bakery,Italian Restaurant,Restaurant,Sandwich Place,Pizza Place,Bagel Shop,Café,Pub
6,Coffee shops everywhere,"CFB Toronto, Downsview East",Clothing Store,Coffee Shop,Athletics & Sports,Pizza Place,American Restaurant,Vietnamese Restaurant,Cosmetics Shop,Gas Station,Sushi Restaurant,Fast Food Restaurant
7,Coffee shops everywhere,Cedarbrae,Coffee Shop,Clothing Store,Gas Station,Pharmacy,Sandwich Place,Gym,Restaurant,Fast Food Restaurant,Wings Joint,Indian Restaurant
8,Ethnic restaurants galore,"Clarks Corners, Sullivan, Tam O'Shanter",Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pharmacy,Bank,Bakery,Sandwich Place,Gas Station,Pizza Place,Falafel Restaurant
9,Coffee shops everywhere,"Cliffcrest, Cliffside, Scarborough Village West",Park,Harbor / Marina,Coffee Shop,Pizza Place,Grocery Store,Beach,Pharmacy,Sandwich Place,Fast Food Restaurant,Bank


#### Ok, Let's look at the venues for each cluster group and make up better descriptions.

In [105]:
cluster0 = "Ethnic restaurants galore"
cluster1 = "Zoo land"
cluster2 = "Coffee shops everywhere"
cluster3 = "Trails and quiet"
cluster4 = "Korea an-nyeong-ha-se-yo"
cluster_labels = [cluster0,cluster1,cluster2,cluster3,cluster4]

"""
clabels= []
for i in kmeans.labels_:
    clabels.append(cluster_labels[i])
print(clabels)
"""

'\nclabels= []\nfor i in kmeans.labels_:\n    clabels.append(cluster_labels[i])\nprint(clabels)\n'

In [101]:
# add clustering labels

try:
    neighbourhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace=True)
except:
    print('No cluster labels')

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) #clabels)

north40_merged = df_north40

# merge north40_grouped with north40_data to add latitude/longitude for each neighborhood
north40_merged = north40_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

north40_merged.head(40) # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,3,Trail,Golf Course,Grocery Store,Health & Beauty Service,Zoo Exhibit,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Stand,Food Court
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,0,Chinese Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Dessert Shop,Gas Station,Vietnamese Restaurant,Korean Restaurant,Bakery,Park
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Zoo Exhibit,Fast Food Restaurant,Zoo,Gas Station,Athletics & Sports,Pizza Place,Restaurant,Caribbean Restaurant,Liquor Store,Grocery Store
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,2,Coffee Shop,Chinese Restaurant,Bank,Pharmacy,Park,Sandwich Place,Japanese Restaurant,Pizza Place,Bakery,Gym
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0,Chinese Restaurant,Coffee Shop,Bakery,Fast Food Restaurant,Sandwich Place,Japanese Restaurant,Park,Pizza Place,Pharmacy,Athletics & Sports
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,0,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Cantonese Restaurant,Indian Restaurant,Bakery,Breakfast Spot,Shopping Mall
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,4,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Bank,Café,Japanese Restaurant,Dessert Shop,Ramen Restaurant,Juice Bar
19,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Park,Coffee Shop,Pharmacy,Bank,Clothing Store,Café,Gas Station,Grocery Store,Liquor Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Breakfast Spot,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Supermarket,Food & Drink Shop,Mexican Restaurant,Mobile Phone Shop,Burger Joint
24,M2R,North York,Willowdale West,43.782736,-79.442259,2,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Park,Korean Restaurant,Convenience Store,Pharmacy,Fast Food Restaurant,Sandwich Place,Grocery Store


Finally, a thousand and two steps later, let's visualize the results.

#### Import the required packages

In [102]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [108]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north40_merged['Latitude'], north40_merged['Longitude'], north40_merged['Neighbourhood'], north40_merged['Cluster Labels']):
    
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    label = folium.Popup(str(poi) + ' Cluster: ' + cluster_labels[cluster], parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### And... It's a beautiful day in the neighboUrhood.